In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

#データセットの読み込み
TRAIN_DATA_PATH: str = "dataset/train_Mod.csv"
TEST_DATA_PATH: str = "dataset/test.csv"
train_data = pd.read_csv(TRAIN_DATA_PATH)
test_data = pd.read_csv(TEST_DATA_PATH)

train_data.head(5)

#欠損値の補完（とりあえず平均値で）
train_data["knee_depth"] = train_data["knee_depth"].fillna(train_data["knee_depth"].mean())
train_data["hip_depth"] = train_data["hip_depth"].fillna(train_data["hip_depth"].mean())

#idはいらないので削除
train_data = train_data.drop("id", axis=1)
test_data = test_data.drop("id", axis=1)
train_data.isnull().sum()

#学習データを訓練用、検証用に分割
train_data, valid_data = train_test_split(train_data, test_size=0.2, random_state=0)
X_train = train_data.drop("personal_id", axis=1)
y_train = train_data["personal_id"]
X_valid = valid_data.drop("personal_id", axis=1)
y_valid = valid_data["personal_id"]

In [2]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

param_grid = {
    'n_estimators': [20, 50, 100],
    'learning_rate': [0.01, 0.1, 0.5],
    'num_leaves': [31, 127],
    'boosting_type' : ['gbdt', 'dart'],
    'objective' : ['regression'],
}
lgb_estimator = lgb.LGBMRegressor()

gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)


#yのラベリング
train_y = train_data["personal_id"]
valid_y = valid_data["personal_id"]
# LabelEncoderのインスタンスを作成
label_encoder = LabelEncoder()
# クラスラベルを数値に変換
y_train_encoded = label_encoder.fit_transform(train_y)
y_valid_encoded = label_encoder.fit_transform(valid_y)

gsearch.fit(X_train, y_train_encoded)

#最適なパラメータを表示
print("the best param is : ", gsearch.best_params_)
print("the best score is : ", gsearch.best_score_)
#検証データで評価
y_pred = gsearch.predict(X_valid)
print("valid accuracy_score : ", accuracy_score(y_valid_encoded, y_pred))

best_lightgbm = gsearch.best_estimator_

Fitting 3 folds for each of 36 candidates, totalling 108 fits


c:\Users\syu\.virtualenvs\Rikkyo-OpenHouseComp-hWWgu0EB\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\syu\.virtualenvs\Rikkyo-OpenHouseComp-hWWgu0EB\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000257 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 819
[LightGBM] [Info] Number of data points in the train set: 489, number of used features: 5
[LightGBM] [Info] Start training from score 2.378323
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets